In [2]:
import pandas as pd
import numpy as np
import sys
sys.path.append('/home/pepijn/Documents/uni/y5/thesis/amalur/amalur-experiments/amalur-factorization')

from datasets.tpc_ai.load import load_usecase_01


/tmp/ipykernel_5967/2532907207.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-02-21 19:18:09.504 | ERROR    | amalur.AmalurMatrix:<module>:20 - USE_MKL set to True, but could not import sparse_dot_mkl. Falling back to USE_MKL=False
Traceback (most recent call last):

  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
           │         │     └ {'__name__': '__main__', '__doc__': 'Entry point for launching an IPython kernel.\n\nThis is separate from the ipykernel pack...
           │         └ <code object <module> at 0x

In [3]:
amalur, T = load_usecase_01("./data")

2024-02-21 19:18:09.520 | INFO     | datasets.tpc_ai.load:load_usecase_01:29 - Loading data for usecase01
2024-02-21 19:18:09.522 | DEBUG    | datasets.tpc_ai.load:load_usecase_01:34 - Start reading order_path='./data/order.csv'
2024-02-21 19:18:09.527 | DEBUG    | datasets.tpc_ai.load:load_usecase_01:36 - Start reading lineitem_path='./data/lineitem.csv'
2024-02-21 19:18:09.529 | DEBUG    | datasets.tpc_ai.load:load_usecase_01:38 - Start reading order_returns_path='./data/order_returns.csv'
2024-02-21 19:18:09.531 | DEBUG    | datasets.tpc_ai.load:load_usecase_01:40 - Finished reading data
2024-02-21 19:18:09.544 | INFO     | datasets.tpc_ai.load:load_usecase_01:67 - Finished preprocessing for usecase10, now creating matrices
2024-02-21 19:18:09.545 | INFO     | datasets.tpc_ai.load:create_AmalurMatrix:113 - Creating Mapping and Indicator matrices for source 0 with 3 rows
2024-02-21 19:18:09.546 | INFO     | datasets.tpc_ai.load:create_AmalurMatrix:113 - Creating Mapping and Indicator

In [4]:
T['or_return_quantity'] = T['or_return_quantity'].astype(int)
display(T)
display(pd.DataFrame(amalur.materialize().todense()))
print("difference: " , np.sum(np.abs((T.values - amalur.materialize().todense()))))

,o_order_id,o_customer_sk,invoice_year,quantity,price,row_price,or_return_quantity
0,1,11,2024,2,20,40,0
1,1,11,2024,1,25,25,0
2,2,12,2024,3,13,39,0
3,3,11,2023,1,10,10,1


,0,1,2,3,4,5,6
0,1.0,11.0,2024.0,2.0,20.0,40.0,0.0
1,1.0,11.0,2024.0,1.0,25.0,25.0,0.0
2,2.0,12.0,2024.0,3.0,13.0,39.0,0.0
3,3.0,11.0,2023.0,1.0,10.0,10.0,1.0


difference:  0.0


In [40]:
def bmatrix(a, label=None):
    """Returns a LaTeX bmatrix

    :a: numpy array
    :returns: LaTeX bmatrix as a string
    """
    if len(a.shape) > 2:
        raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    if label in ["T", "S"]:
        rv = [r'\begin{bNiceMatrix}[first-row,last-col]']
        rv += ['&'.join(['C' for x in range(a.shape[1])]) + r' \\']        
    elif label in ["I", "M"]:
        rv = [r'\begin{bNiceMatrix}[last-col]']
    else:
        rv = [r'\begin{bNiceMatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
        
        
    rv +=  [r'\end{bNiceMatrix}']
    return '\n'.join(rv)


In [30]:
def print_as_latex(matrices, label=None):
        for i, x in enumerate(matrices):
            printlabel = label
            x = x.astype(int)
            if label != "T":
                try:
                    x = x.todense()
                except:
                    pass
                printlabel = f"{label}_{i}"
            print("\\begin{equation}" + f"{printlabel} = {bmatrix(x, label)}" + "\\end{equation}")
            
def print_matrices(amalur):           
    print_as_latex([T.values], "T")
    print_as_latex(amalur.S, "S")
    print_as_latex(amalur.I, "I")
    print_as_latex(amalur.M, "M")
print_matrices(amalur)

\begin{equation}T = \begin{bNiceMatrix}[first-row,last-col]
C&C&C&C&C&C&C \\
  1 & 11 & 2024 & 2 & 20 & 40 & 0& L \\
  1 & 11 & 2024 & 1 & 25 & 25 & 0& L \\
  2 & 12 & 2024 & 3 & 13 & 39 & 0& L \\
  3 & 11 & 2023 & 1 & 10 & 10 & 1& L \\
\end{bNiceMatrix}\end{equation}
\begin{equation}S_0 = \begin{bNiceMatrix}[first-row,last-col]
C&C&C \\
  1 & 11 & 2024& L \\
  2 & 12 & 2024& L \\
  3 & 11 & 2023& L \\
\end{bNiceMatrix}\end{equation}
\begin{equation}S_1 = \begin{bNiceMatrix}[first-row,last-col]
C&C&C \\
  2 & 20 & 40& L \\
  1 & 25 & 25& L \\
  3 & 13 & 39& L \\
  1 & 10 & 10& L \\
\end{bNiceMatrix}\end{equation}
\begin{equation}S_2 = \begin{bNiceMatrix}[first-row,last-col]
C \\
  1& L \\
\end{bNiceMatrix}\end{equation}
\begin{equation}I_0 = \begin{bNiceMatrix}[last-col]
  1 & 0 & 0& L \\
  1 & 0 & 0& L \\
  0 & 1 & 0& L \\
  0 & 0 & 1& L \\
\end{bNiceMatrix}\end{equation}
\begin{equation}I_1 = \begin{bNiceMatrix}[last-col]
  1 & 0 & 0 & 0& L \\
  0 & 1 & 0 & 0& L \\
  0 & 0 & 1 & 0& L

In [53]:
Ts = []
for i in range(3):
    Ts.append(amalur.I[i] * amalur.S[i] * amalur.M[i].T.todense())
    print_as_latex([Ts[i]], 'T')

\begin{equation}T = \begin{bNiceMatrix}[first-row,last-col]
C&C&C&C&C&C&C \\
  1 & 11 & 2024 & 0 & 0 & 0 & 0\\
  1 & 11 & 2024 & 0 & 0 & 0 & 0\\
  2 & 12 & 2024 & 0 & 0 & 0 & 0\\
  3 & 11 & 2023 & 0 & 0 & 0 & 0\\
\end{bNiceMatrix}\end{equation}
\begin{equation}T = \begin{bNiceMatrix}[first-row,last-col]
C&C&C&C&C&C&C \\
  0 & 0 & 0 & 2 & 20 & 40 & 0\\
  0 & 0 & 0 & 1 & 25 & 25 & 0\\
  0 & 0 & 0 & 3 & 13 & 39 & 0\\
  0 & 0 & 0 & 1 & 10 & 10 & 0\\
\end{bNiceMatrix}\end{equation}
\begin{equation}T = \begin{bNiceMatrix}[first-row,last-col]
C&C&C&C&C&C&C \\
  0 & 0 & 0 & 0 & 0 & 0 & 0\\
  0 & 0 & 0 & 0 & 0 & 0 & 0\\
  0 & 0 & 0 & 0 & 0 & 0 & 0\\
  0 & 0 & 0 & 0 & 0 & 0 & 1\\
\end{bNiceMatrix}\end{equation}


In [8]:
print_as_latex([(amalur.I[0] * amalur.S[0]).todense()], 'T')

\begin{equation}T = \begin{bNiceMatrix}[first-row,last-col]
C&C&C \\
  1 & 11 & 2024& L \\
  1 & 11 & 2024& L \\
  2 & 12 & 2024& L \\
  3 & 11 & 2023& L \\
\end{bNiceMatrix}\end{equation}


In [9]:
print_as_latex([amalur.M[0].T], 'M')

\begin{equation}M_0 = \begin{bNiceMatrix}[last-col]
  1 & 0 & 0 & 0 & 0 & 0 & 0& L \\
  0 & 1 & 0 & 0 & 0 & 0 & 0& L \\
  0 & 0 & 1 & 0 & 0 & 0 & 0& L \\
\end{bNiceMatrix}\end{equation}


## Running Example: Right Matrix Multiplication

In [45]:
print(bmatrix(T.values))

\begin{bNiceMatrix}
  1 & 11 & 2024 & 2 & 20 & 40 & 0\\
  1 & 11 & 2024 & 1 & 25 & 25 & 0\\
  2 & 12 & 2024 & 3 & 13 & 39 & 0\\
  3 & 11 & 2023 & 1 & 10 & 10 & 1\\
\end{bNiceMatrix}


In [49]:
X = np.array([1,1,2,3])
print(bmatrix(X))

\begin{bNiceMatrix}
  1 & 1 & 2 & 3\\
\end{bNiceMatrix}


In [50]:
print(bmatrix(X.dot(T.values)))

\begin{bNiceMatrix}
  15 & 79 & 14165 & 12 & 101 & 173 & 3\\
\end{bNiceMatrix}


In [54]:
for T in Ts:
    print(bmatrix(X.dot(T)))

\begin{bNiceMatrix}
  15. & 79. & 14165. & 0. & 0. & 0. & 0.\\
\end{bNiceMatrix}
\begin{bNiceMatrix}
  0. & 0. & 0. & 12. & 101. & 173. & 0.\\
\end{bNiceMatrix}
\begin{bNiceMatrix}
  0. & 0. & 0. & 0. & 0. & 0. & 3.\\
\end{bNiceMatrix}


In [56]:
sum(Ts)

matrix([[1.000e+00, 1.100e+01, 2.024e+03, 2.000e+00, 2.000e+01,
         4.000e+01, 0.000e+00],
        [1.000e+00, 1.100e+01, 2.024e+03, 1.000e+00, 2.500e+01,
         2.500e+01, 0.000e+00],
        [2.000e+00, 1.200e+01, 2.024e+03, 3.000e+00, 1.300e+01,
         3.900e+01, 0.000e+00],
        [3.000e+00, 1.100e+01, 2.023e+03, 1.000e+00, 1.000e+01,
         1.000e+01, 1.000e+00]])